# Working with Queries and Scans

## Query

The `Query` operation finds items based on primary key values. Partition key must be provided as equal condition and optionally sort key can be provided in `KeyConditionExpression` parameter to refine the results. Additionally you use `FilterExpression` and `Limit` to narrow down the results.

The number of capacity units consumed will be

- the same whether you request all of the attributes (the default behavior) or just some of them (using a `ProjectionExpression`)
- the same whether or not you use a `FilterExpression`

You can use `Query` from table and indexes as well. To use an index for querying, `IndexName` should be provided. If not, DynamoDB always fetch data from the base table.

In addition to the items that match your criteria, the Query response contains the following elements: 

- ScannedCount is the number of items that matched the key condition expression before a filter expression (if present) was applied
- Count is the number of items that remain after a filter expression (if present) was applied

Consumed capacity unit is calculated based on Scanned Count, not Count.

In [8]:
# import and get dynamodb resource
import boto3
from boto3.dynamodb.conditions import Key, Attr
from botocore.exceptions import ClientError
from pprint import pprint
from decimal import Decimal

dynamodb = boto3.resource('dynamodb')
starbucks = dynamodb.Table('Starbucks')

In [32]:
# search all branches in Korea
response = starbucks.query(
    IndexName='GSI_01_Locations',
    KeyConditionExpression='Country = :country',
    Limit=5,
    ExpressionAttributeValues={
        ':country': 'KR'
    },
    ReturnConsumedCapacity='INDEXES'
)

pprint(response)

{'ConsumedCapacity': {'CapacityUnits': 0.5,
                      'GlobalSecondaryIndexes': {'GSI_01_Locations': {'CapacityUnits': 0.5}},
                      'Table': {'CapacityUnits': 0.0},
                      'TableName': 'Starbucks'},
 'Count': 5,
 'Items': [{'Brand': 'Starbucks',
            'City': 'Gyeonggido',
            'Country': 'KR',
            'Latitude': Decimal('37.39'),
            'Longitude': Decimal('126.46'),
            'OwnershipType': 'Joint Venture',
            'Postcode': '410-380',
            'State': '11',
            'StateCity': '11::Gyeonggido',
            'StoreName': 'Lafesta Square',
            'StoreNumber': '20644-207432',
            'StreetAddress': '770-1, Janghangdong, Ilsandonggu, Goyangshi, '
                             'Gyeonnggido',
            'Timezone': 'GMT+09:00 Asia/Seoul'},
           {'Brand': 'Starbucks',
            'City': 'Seoul',
            'Country': 'KR',
            'Latitude': Decimal('37.52'),
            'Longitud

In [33]:
# search all branches in Korea, which contains Seocho in StreetAddress attribute
# Check Count and ScannedCount in the result
response = starbucks.query(
    IndexName='GSI_01_Locations',
    KeyConditionExpression='Country = :country',
    Limit=50,
    FilterExpression='contains(StreetAddress, :addr)',
    ExpressionAttributeValues={
        ':country': 'KR',
        ':addr': 'Seocho'
    },
    ReturnConsumedCapacity='INDEXES'
)

pprint(response)

{'ConsumedCapacity': {'CapacityUnits': 1.5,
                      'GlobalSecondaryIndexes': {'GSI_01_Locations': {'CapacityUnits': 1.5}},
                      'Table': {'CapacityUnits': 0.0},
                      'TableName': 'Starbucks'},
 'Count': 3,
 'Items': [{'Brand': 'Starbucks',
            'City': 'Seoul',
            'Country': 'KR',
            'Latitude': Decimal('37.51'),
            'Longitude': Decimal('127.02'),
            'OwnershipType': 'Joint Venture',
            'Postcode': '137-810',
            'State': '11',
            'StateCity': '11::Seoul',
            'StoreName': 'Sinnonhyeon Station',
            'StoreNumber': '23179-228325',
            'StreetAddress': '748-1, banpo-dong, Seocho-gu',
            'Timezone': 'GMT+09:00 Asia/Seoul'},
           {'Brand': 'Starbucks',
            'City': 'Seoul',
            'Country': 'KR',
            'Latitude': Decimal('37.49'),
            'Longitude': Decimal('126.99'),
            'OwnershipType': 'Joint Ventur

## Pagination

DynamoDB paginates the results from `Query` operations. With pagination, the Query results are divided into pages of data that are 1 MB in size (or less). An application can process the first page of results, then the second page, and so on. You consider this as a simple cursor.

To retrieve all pages, your application should do the following.

- Run a `Query` operation
- If the result contains a `LastEvaluatedKey` element and it is not empty, take the `LastEvaluatedKey` and set it as `ExclusiveStartKey` parameter in the new `Query` statement
- Run the new statement until `LastEvaluatedKey` is empty

In [35]:
# it contains LastEvaluatedKey, so set it ExclusiveStartKey in the next cell
response = starbucks.query(
    IndexName='GSI_01_Locations',
    KeyConditionExpression='Country = :country',
    Limit=50,
    FilterExpression='contains(StreetAddress, :addr)',
    ExpressionAttributeValues={
        ':country': 'KR',
        ':addr': 'Seocho'
    },
    ReturnConsumedCapacity='INDEXES'
)

pprint(response)

{'ConsumedCapacity': {'CapacityUnits': 1.5,
                      'GlobalSecondaryIndexes': {'GSI_01_Locations': {'CapacityUnits': 1.5}},
                      'Table': {'CapacityUnits': 0.0},
                      'TableName': 'Starbucks'},
 'Count': 3,
 'Items': [{'Brand': 'Starbucks',
            'City': 'Seoul',
            'Country': 'KR',
            'Latitude': Decimal('37.51'),
            'Longitude': Decimal('127.02'),
            'OwnershipType': 'Joint Venture',
            'Postcode': '137-810',
            'State': '11',
            'StateCity': '11::Seoul',
            'StoreName': 'Sinnonhyeon Station',
            'StoreNumber': '23179-228325',
            'StreetAddress': '748-1, banpo-dong, Seocho-gu',
            'Timezone': 'GMT+09:00 Asia/Seoul'},
           {'Brand': 'Starbucks',
            'City': 'Seoul',
            'Country': 'KR',
            'Latitude': Decimal('37.49'),
            'Longitude': Decimal('126.99'),
            'OwnershipType': 'Joint Ventur

In [36]:
# you should repeat this until LastEvaluatedKey is empty
response = starbucks.query(
    IndexName='GSI_01_Locations',
    KeyConditionExpression='Country = :country',
    Limit=50,
    FilterExpression='contains(StreetAddress, :addr)',
    ExpressionAttributeValues={
        ':country': 'KR',
        ':addr': 'Seocho'
    },
    ExclusiveStartKey={
        'Country': 'KR',
        'StateCity': '11::Seoul',
        'StoreNumber': '17830-186444'
    },
    ReturnConsumedCapacity='INDEXES'
)

pprint(response)

{'ConsumedCapacity': {'CapacityUnits': 1.5,
                      'GlobalSecondaryIndexes': {'GSI_01_Locations': {'CapacityUnits': 1.5}},
                      'Table': {'CapacityUnits': 0.0},
                      'TableName': 'Starbucks'},
 'Count': 10,
 'Items': [{'Brand': 'Starbucks',
            'City': 'Seoul',
            'Country': 'KR',
            'Latitude': Decimal('37.49'),
            'Longitude': Decimal('127.01'),
            'OwnershipType': 'Joint Venture',
            'Postcode': '137-883',
            'State': '11',
            'StateCity': '11::Seoul',
            'StoreName': 'Gyodae Station',
            'StoreNumber': '23332-229045',
            'StreetAddress': '1693-3, Seocho-dong, Seocho-gu',
            'Timezone': 'GMT+09:00 Asia/Seoul'},
           {'Brand': 'Starbucks',
            'City': 'Seoul',
            'Country': 'KR',
            'Latitude': Decimal('37.5'),
            'Longitude': Decimal('126.99'),
            'OwnershipType': 'Joint Venture',

## Scan

> Never use the `Scan` operation if you know why you're using `Scan` operation

`Scan` operation operates on the entire table - full table scan. Even though you use `FilterExpression`, basically all items in the table are scanned first (also consumes RCU) and then applied filter expressions. Maybe if you need to do these kinds of jobs, `Scan` help you to achieve it.

- Very small table, e.g. code table
- Migration to another database

A `Scan` operation has the same limitations as we've seen.

- A `Scan` operation can retrieve a maximum of 1 MB of data which of limit applies before the filter expression
- A `Limit` parameter limits the number of items that it returns to the result, but applies before the filter expression
- Pagination is needed if the result is enough to be paged
- `ScannedCount` and `Count` can be different
- A `Scan` operation performs eventually consistent reads, by default
- `ProjectionExpression` and `FilterExpression` could help to reduce the network traffic, not consumed capacity units

In [40]:
# it returns all the items in the table, but up to 1 MB with LastEvaluatedKey
response = starbucks.scan()

pprint(response, depth=1)

{'Count': 3847,
 'Items': [...],
 'LastEvaluatedKey': {...},
 'ResponseMetadata': {...},
 'ScannedCount': 3847}


In [43]:
# it returns Count=0, ScannedCount=100, LastEvaluatedKey={'StoreNumber': '12004-10855'}
response = starbucks.scan(
    FilterExpression='contains(StreetAddress, :addr)',
    Limit=100,
    ExpressionAttributeValues={
        ':addr': 'Shamkha'
    }
)

pprint(response, depth=2)

{'Count': 0,
 'Items': [],
 'LastEvaluatedKey': {'StoreNumber': '12004-10855'},
 'ResponseMetadata': {'HTTPHeaders': {...},
                      'HTTPStatusCode': 200,
                      'RequestId': '1VQ2G1P098RKT2465OBGNVN7SVVV4KQNSO5AEMVJF66Q9ASUAAJG',
                      'RetryAttempts': 0},
 'ScannedCount': 100}


In some cases, scanning all existing data is needed. If you have a large amount of data, scanning all data through a table can take quite a while. To alleviate this, DynamoDB provides parallel scan. When making a `Scan`, you can set `TotalSegments` and `Segments` parameters to divide the table into multiple segments and do the parallel jobs.

- `Segment` being scanned by a particular worker
- `TotalSegment` is the total number of segments for the parallel scan

In [44]:
# it will scan with 10 threads
# in real case, each thread should have pagination logic
responses = []
number_of_threads = 10

for thread in range(number_of_threads):
    responses.append(
        starbucks.scan(
            TotalSegments=number_of_threads,
            Segment=thread
        )
    )

pprint(responses, depth=2)

total_count = sum([r['Count'] for r in responses])
print('\nTotal count is', total_count)

[{'Count': 2505,
  'Items': [...],
  'ResponseMetadata': {...},
  'ScannedCount': 2505},
 {'Count': 2541,
  'Items': [...],
  'ResponseMetadata': {...},
  'ScannedCount': 2541},
 {'Count': 2595,
  'Items': [...],
  'ResponseMetadata': {...},
  'ScannedCount': 2595},
 {'Count': 2527,
  'Items': [...],
  'ResponseMetadata': {...},
  'ScannedCount': 2527},
 {'Count': 2546,
  'Items': [...],
  'ResponseMetadata': {...},
  'ScannedCount': 2546},
 {'Count': 2625,
  'Items': [...],
  'ResponseMetadata': {...},
  'ScannedCount': 2625},
 {'Count': 2530,
  'Items': [...],
  'ResponseMetadata': {...},
  'ScannedCount': 2530},
 {'Count': 2621,
  'Items': [...],
  'ResponseMetadata': {...},
  'ScannedCount': 2621},
 {'Count': 2549,
  'Items': [...],
  'ResponseMetadata': {...},
  'ScannedCount': 2549},
 {'Count': 2563,
  'Items': [...],
  'ResponseMetadata': {...},
  'ScannedCount': 2563}]

Total count is 25602
